In [52]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd


options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Otwieramy stronę z danymi meczu
driver.get("https://www.flashscore.pl/pilka-nozna/polska/pko-bp-ekstraklasa-2013-2014/wyniki/")

# Czekamy, aż elementy zostaną załadowane
wait = WebDriverWait(driver, 10)
while True:
    try:
        more_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "event__more--static")))
        more_button.click()
        wait.until(EC.invisibility_of_element_located((By.CLASS_NAME, "preloader__animation")))
    except:
        break

data = pd.DataFrame(columns=['Home','Away','Home score', 'Away score', 'Posiadanie piłki H', 'Sytuacje bramkowe H', 'Strzały na bramkę H', 'Strzały niecelne H', 'Strzały zablokowane H', 'Rzuty wolne H', 'Rzuty rożne H', 'Spalone H', 'Wrzuty z autu H', 'Interwencje bramkarzy H', 'Faule H', 'Czerwone kartki H', 'Żółte kartki H', 'Posiadanie piłki A', 'Sytuacje bramkowe A', 'Strzały na bramkę A', 'Strzały niecelne A', 'Strzały zablokowane A', 'Rzuty wolne A', 'Rzuty rożne A', 'Spalone A', 'Wrzuty z autu A', 'Interwencje bramkarzy A', 'Faule A', 'Czerwone kartki A', 'Żółte kartki A'])

for match in wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "event__match"))):
    match.click()
    wait.until(EC.number_of_windows_to_be(2))
    driver.switch_to.window(driver.window_handles[1])
    stats_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a[href='#/szczegoly-meczu/statystyki-meczu']")))
    stats_button.click()
    categories = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "stat__categoryName")))
    team_names = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".participant__participantName")))

    row = []
    team_names = list(set([team.text for team in team_names]))
    for i in team_names:
        row.append(i)

    for i in categories:
        match i.text:
            case 'Posiadanie piłki':
                

    # home_team = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "stat__homeValue")))
    # away_team = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "stat__awayValue")))
    # stats = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[class='stat__categoryName']")))
    # team_names = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".participant__participantName")))

    # # sprawdzamy, czy wśród znalezionych elementów jest taki z tekstem "Czerwone kartki"
    # team_names = list(set([team.text for team in team_names]))
    # for i in team_names:
    #     row.append(i)
    # for i in home_team:
    #     row.append(i.text)
    # for i in away_team:
    #     row.append(i.text)

    # for stat in stats:
    #     if stat.text == "Czerwone kartki":
    #         print("Znaleziono statystykę czerwonych kartek.")
    #         break
    # else:
    #     row.insert(13, '0')
    #     row.insert(26, '0')
    #     print("Nie znaleziono statystyki czerwonych kartek.")

    # print(row)
    # new_df = pd.DataFrame([row], columns=data.columns)
    # data = pd.concat([data, new_df], ignore_index=True)

    # display(data)
    driver.close()
    driver.switch_to.window(driver.window_handles[0])


# home_team = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "event__participant--home")))
# away_team = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "event__participant--away")))

# # Wypisujemy znalezione elementy
# for i in home_team:
#     print("Drużyna gospodarzy: " + i.text)


# Zamykamy przeglądarkę
driver.quit()


Posiadanie piłki
aaaaaaa
Sytuacje bramkowe
Strzały na bramkę
Strzały niecelne
Strzały zablokowane
Rzuty wolne
Rzuty rożne
Spalone
Wrzuty z autu
Interwencje bramkarzy
Faule
Żółte kartki
Posiadanie piłki
aaaaaaa
Sytuacje bramkowe
Strzały na bramkę
Strzały niecelne
Strzały zablokowane
Rzuty wolne
Rzuty rożne
Spalone
Wrzuty z autu
Interwencje bramkarzy
Faule
Żółte kartki
Posiadanie piłki
aaaaaaa
Sytuacje bramkowe
Strzały na bramkę
Strzały niecelne
Strzały zablokowane
Rzuty wolne
Rzuty rożne
Spalone
Wrzuty z autu
Interwencje bramkarzy
Faule
Żółte kartki


KeyboardInterrupt: 